In [19]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os

In [20]:
num_classes = 2
img_rows,img_cols = 48,48
batch_size = 32

In [21]:
train_data_dir = './input/train'
validation_data_dir = './input/val'

In [22]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 30,
    shear_range = 0.3,
    zoom_range = 0.3,
    width_shift_range = 0.4,
    height_shift_range = 0.4,
    horizontal_flip = True,
    fill_mode = 'nearest'
)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [23]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode = 'grayscale',
    target_size = (img_rows,img_cols),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = True
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    color_mode = 'grayscale',
    target_size = (img_rows,img_cols),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = True
)

Found 1434 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [24]:
model = Sequential()
model.add(Conv2D(32,(3,3),padding = 'same',kernel_initializer = 'he_normal',input_shape = (img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding = 'same',kernel_initializer = 'he_normal',input_shape = (img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),padding = 'same',kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding = 'same',kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128,(3,3),padding = 'same',kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding = 'same',kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256,(3,3),padding = 'same',kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding = 'same',kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(64,kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(num_classes,kernel_initializer = 'he_normal'))
model.add(Activation('softmax'))

In [25]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
activation_11 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_12 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 24, 24, 32)       

In [26]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [31]:
checkpoint = ModelCheckpoint('StrokeDetectionModel.h5',
                             monitor = 'val_accuracy',
                             mode = 'max',
                             save_best_only = True,
                             verbose = 1
                            )
earlystop = EarlyStopping(monitor = 'val_accuracy',
                          min_delta = 0,
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True
                          )
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy',
                              factor = 0.2,
                              patience = 3,
                              verbose = 1,
                              min_delta = 0.0001)
callbacks = [earlystop,checkpoint,reduce_lr]

In [32]:
model.compile(loss = 'categorical_crossentropy',optimizer = Adam(learning_rate = 0.001),metrics = ['accuracy'])
nb_train_samples = 717
nb_validation_samples = 307
epochs = 20
history = model.fit(train_generator,steps_per_epoch = nb_train_samples//batch_size,epochs = epochs,callbacks = callbacks,validation_data = validation_generator,validation_steps = nb_validation_samples//batch_size)

Epoch 1/20
22/22 [==============================] - 120s 5s/step - loss: 0.7435 - accuracy: 0.6512 - val_loss: 1.3353 - val_accuracy: 0.5972

Epoch 00001: val_accuracy improved from -inf to 0.59722, saving model to StrokeDetectionModel.h5
Epoch 2/20
22/22 [==============================] - 96s 4s/step - loss: 0.7571 - accuracy: 0.6416 - val_loss: 1.3433 - val_accuracy: 0.5694

Epoch 00002: val_accuracy did not improve from 0.59722
Epoch 3/20
22/22 [==============================] - 112s 5s/step - loss: 0.6791 - accuracy: 0.6567 - val_loss: 1.1952 - val_accuracy: 0.5694

Epoch 00003: val_accuracy did not improve from 0.59722
Epoch 4/20
22/22 [==============================] - 127s 6s/step - loss: 0.7162 - accuracy: 0.6472 - val_loss: 1.2212 - val_accuracy: 0.5278
Restoring model weights from the end of the best epoch.

Epoch 00004: val_accuracy did not improve from 0.59722

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 00004: early stopping


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [13]:
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np

In [14]:
face_classifier = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
classifier = load_model('./StrokeDetectionModel.h5')

In [15]:
class_labels = ['Asymmetrical','Normal']
cap = cv2.VideoCapture(0)

In [16]:
while True:
    ret,frame = cap.read()
    labels = []
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.3,5)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

            preds = classifier.predict(roi)[0]
            label = class_labels[preds.argmax()]
            label_position = (x,y)
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
        else:
            cv2.putText(frame,'No Face Found',(20,20),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    
    cv2.imshow('Stroke Detector',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()